# Imports

In [7]:
import nltk
nltk.download('stopwords')
import numpy as np

from nltk.book import *
from string import punctuation
from collections import Counter

from pprint import pprint # get some prettier printing of objects

from nltk.corpus import stopwords

sw = stopwords.words('english')

import sqlite3
from collections import Counter, defaultdict
from string import punctuation

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aidan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# SQLite Connection and Query

In [4]:
convention_db = sqlite3.connect("ConventionSpeeches.db")
convention_cur = convention_db.cursor()

In [5]:
query_results = convention_cur.execute(
                            '''
                                SELECT text, party
                                FROM conventions
                                WHERE speaker != "Unknown"
                            ''')

In [8]:
convention_data = defaultdict(str)

for row in query_results :
    text, party = row

    # Clean text
    text = "".join([ch for ch in text if ch not in punctuation])    
    text = [w.lower() for w in text.split() if w.isalpha()]
    
    convention_data[party] += " ".join(text) + " "

# Function Creation

In [139]:
def the_function(corpus_1,corpus_2,num_words,ratio_cutoff):
    #input_list = (corpus_1,corpus_2)
    combined_dict1 = {}
    combined_dict2 = {}
    oneV2Dict = {}
    twoV1Dict = {}
    outputDict = {"one":combined_dict1,"two":combined_dict2,"one_vs_two": oneV2Dict,"two_vs_one":twoV1Dict}
    
    c = Counter()
    
    total_tokens = 1
    unique_tokens = 0
    avg_token_len = 0.0
    lex_diversity = 0.0
    top_10 = Counter()
    
    #Summary Stats for 1
    c1_clean = [word for word in corpus_1]
    c1_clean = [word.lower() for word in c1_clean if word.isalpha() and word.lower() not in sw]
    
    
    c.update(c1_clean)
        
     # Calculate your statistics here
    total_tokens = len(c1_clean)
    unique_tokens = len(set(c1_clean))
    lex_diversity = (unique_tokens/total_tokens)
    
    
    token_len = [len(word) for word in c1_clean]
    
    avg_token_len = np.mean(token_len)
    top_10 = c.most_common(10)
    
    combined_dict1 = {'tokens':total_tokens,
                'unique_tokens':unique_tokens,
                'avg_token_length':avg_token_len,
                'lexical_diversity':lex_diversity,
                'top_words':top_10}
    
    #Summary Stats for 2
    c2_clean = [word for word in corpus_2]
    c2_clean = [word.lower() for word in c2_clean if word.isalpha() and word.lower() not in sw]
    
    
    c.update(c2_clean)
        
     # Calculate your statistics here
    total_tokens = len(c2_clean)
    unique_tokens = len(set(c2_clean))
    lex_diversity = (unique_tokens/total_tokens)
    
    
    token_len = [len(word) for word in c2_clean]
    
    avg_token_len = np.mean(token_len)
    top_10 = c.most_common(10)
    
        # Now we'll fill out the dictionary. 
    combined_dict2 = {'tokens':total_tokens,
                'unique_tokens':unique_tokens,
                'avg_token_length':avg_token_len,
                'lexical_diversity':lex_diversity,
                'top_words':top_10}
    
    freq1 = nltk.FreqDist(c1_clean)
    freq2 = nltk.FreqDist(c2_clean)
    
    corpus_one_concentration = defaultdict(float)
    corpus_two_concentration = defaultdict(float)
    
    for word, freq in freq1.items():
        corpus_one_concentration[word] = freq/len(c1_clean)
    
    for word, freq in freq2.items():
        corpus_two_concentration[word] = freq/len(c2_clean)
        
    
    
    oneV2Dict = {}
    twoV1Dict = {}
    
    for word in corpus_one_concentration:
        if ratio_cutoff > freq1[word] or ratio_cutoff > freq2[word]:
            continue
        ratio1 = corpus_one_concentration[word]/corpus_two_concentration[word] 
        
        oneV2Dict[word] = ratio1
    
    for word in corpus_two_concentration:
        if ratio_cutoff > freq1[word] or ratio_cutoff > freq2[word]:
            continue
        ratio2 = corpus_two_concentration[word]/corpus_one_concentration[word] 
        
        twoV1Dict[word] = ratio2
        
    
    
    
    #need to figure out how to limit to num_words for one_v2 and two_vone
    outputDict = {"one":combined_dict1,"two":combined_dict2,"one_vs_two": sorted(oneV2Dict.items(), key = lambda item:item[1], reverse = True)[:num_words],"two_vs_one":sorted(twoV1Dict.items(), key = lambda item:item[1], reverse = True)[:num_words]}
    print(outputDict)

In [153]:
d_split = convention_data['Democratic'].split()
r_split = convention_data['Republican'].split()
big = open("big.txt",'r').read()
big = big.split()

# Results

In [154]:
the_function(r_split, d_split,10,5)

{'one': {'tokens': 154088, 'unique_tokens': 13158, 'avg_token_length': 6.132599553501895, 'lexical_diversity': 0.0853927625772286, 'top_words': [('president', 1699), ('america', 1504), ('us', 1195), ('people', 1009), ('american', 986), ('country', 985), ('trump', 878), ('know', 829), ('thank', 796), ('years', 787)]}, 'two': {'tokens': 238698, 'unique_tokens': 15011, 'avg_token_length': 6.0489656385893475, 'lexical_diversity': 0.0628869952827422, 'top_words': [('president', 4133), ('america', 3856), ('us', 3283), ('people', 2615), ('country', 2306), ('american', 2219), ('know', 2082), ('john', 2040), ('one', 2018), ('years', 1860)]}, 'one_vs_two': [('radical', 15.835262966616478), ('media', 15.684655846010072), ('sarah', 11.360079954311823), ('gulf', 11.360079954311823), ('islamic', 9.552794507034942), ('usa', 8.606121177508955), ('prayers', 7.74550905975806), ('ann', 6.712774518456985), ('deals', 6.368529671356627), ('regulations', 6.337234685256596)], 'two_vs_one': [('edwards', 38.861

In [155]:
the_function(r_split,big,10,5)

{'one': {'tokens': 154088, 'unique_tokens': 13158, 'avg_token_length': 6.132599553501895, 'lexical_diversity': 0.0853927625772286, 'top_words': [('president', 1699), ('america', 1504), ('us', 1195), ('people', 1009), ('american', 986), ('country', 985), ('trump', 878), ('know', 829), ('thank', 796), ('years', 787)]}, 'two': {'tokens': 410950, 'unique_tokens': 25173, 'avg_token_length': 6.361155858376931, 'lexical_diversity': 0.06125562720525611, 'top_words': [('one', 3652), ('said', 3304), ('may', 2642), ('would', 2477), ('could', 1948), ('president', 1938), ('prince', 1713), ('us', 1650), ('american', 1648), ('america', 1642)]}, 'one_vs_two': [('applause', 388.3126525102539), ('tonight', 126.87216766671364), ('job', 93.72538512315784), ('bless', 82.94315585898967), ('cheers', 65.78556841977745), ('values', 54.67314132184206), ('lets', 45.71970006007699), ('communities', 43.11621713998927), ('economy', 37.651517696533986), ('democrat', 35.81376504706031)], 'two_vs_one': [('french', 62.